# Creating and tracking XGBoost hyperparameter experiments with MLflow and Optuna

## Imports

In [1]:
#----------------------------------------- packages
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder

import xgboost as xgb

import optuna
import mlflow

from pprint import pformat
#----------------------------------------- settings
pd.set_option('display.max_columns', None)

C:\Users\RhysKilian\anaconda3\envs\xgboost-scikit-mlflow-optuna\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
C:\Users\RhysKilian\anaconda3\envs\xgboost-scikit-mlflow-optuna\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data preparation
We use the [Telco Customer Churn](https://www.kaggle.com/datasets/blastchar/telco-customer-churn) available on Kaggle which is a binary classification problem. The data set includes information about:
* Customers who left within the last month – the column is called Churn
* Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies
* Customer account information – how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges
* Demographic info about customers – gender, age range, and if they have partners and dependents

### Data processing
There are several Kaggle notebooks that demonstrate how to prepare the Telco churn data. I use the workbook provided by [Kaan Boke](https://www.kaggle.com/code/kaanboke/xgboost-lightgbm-catboost-imbalanced-data) for simplicity. I will not do any data transformation or normalisation, however, you could easily add it in this section.

In [2]:
# Read data
df = pd.read_csv('data/telco_customer_churn.csv')

# Remove columns we are not going to use
df = df.drop(['customerID','gender','PhoneService'],axis=1)

# 1/0 encode the dependent variable
le = LabelEncoder()
df['Churn']=le.fit_transform(df['Churn'])

# Missing data
df['TotalCharges']= df['TotalCharges'].apply(lambda x: x if x!= ' ' else np.nan).astype(float)
df[['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']]= df[['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']].replace('No internet service','No')

# Get dummy variables
df = pd.get_dummies(df)

# X and y arrays before train test split
X = df.drop('Churn', axis=1)
y = df['Churn']

In [3]:
X.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_Yes,TechSupport_No,TechSupport_Yes,StreamingTV_No,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,1,29.85,29.85,0,1,1,0,0,1,0,1,0,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,1,0
1,0,34,56.95,1889.50,1,0,1,0,1,0,0,1,0,0,0,1,1,0,0,1,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1
2,0,2,53.85,108.15,1,0,1,0,1,0,0,1,0,0,0,1,0,1,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,1
3,0,45,42.30,1840.75,1,0,1,0,0,1,0,1,0,0,0,1,1,0,0,1,0,1,1,0,1,0,0,1,0,1,0,1,0,0,0
4,0,2,70.70,151.65,1,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,1,0


In [4]:
y.head()

0    0
1    0
2    1
3    0
4    1
Name: Churn, dtype: int32

### Train test split
We use a 80/20 split.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
X_train

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_Yes,TechSupport_No,TechSupport_Yes,StreamingTV_No,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
2142,0,21,64.85,1336.80,1,0,0,1,1,0,0,1,0,0,0,1,1,0,0,1,1,0,1,0,0,1,0,1,0,1,0,0,0,0,1
1623,0,54,97.20,5129.45,1,0,1,0,0,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,1,1,0,0,0
6074,0,1,23.45,23.45,0,1,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,1,0
1362,0,4,70.20,237.95,1,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,1,0
6754,0,0,61.90,NaN,1,0,0,1,0,0,1,1,0,0,0,1,0,1,1,0,0,1,1,0,1,0,0,0,1,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,0,1,95.00,95.00,0,1,1,0,1,0,0,0,1,0,0,1,1,0,1,0,1,0,0,1,0,1,1,0,0,0,1,0,0,1,0
5191,0,23,91.10,2198.30,0,1,0,1,0,0,1,1,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0
5226,0,12,21.15,306.05,0,1,0,1,1,0,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,1,0
5390,1,12,99.45,1200.15,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,1,1,0,0,1,0,1,1,0,0,0,1,0,0,1,0


## Modelling

### Obtain hyperparameters
Obtain hyperparameters for this trial

In [7]:
# def suggest_hyperparameters(trial):
    
#     # Obtain the learning rate on a logarithmic scale
#     lr = trial.suggest_loguniform('learning_rate', 0.05, 0.5)
    
#     # Obtain the max depth of trees in a range from 3 to 9 with step size 2
#     max_depth = trial.suggest_float('max_depth', 3, 9, step=2)

#     print(f"Suggested hyperparameters: \n{pformat(trial.params)}")
    
#     return lr, max_depth

### Objective

We use the f1 score maximised.

In [8]:
def objective(trial):
    
    print("\n********************************\n")
    
    # Parameters to search over
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 0, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 6),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0,6),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0,2),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.05, 0.5),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.4, 0.9),
        'subsample': trial.suggest_uniform('subsample', 0.4, 0.9),
        'nthread': -1
    }
    
    # Start a new mlflow run
    with mlflow.start_run():
        
        # Get hyperparameter suggestions created by optuna and log them as params using mlflow
        # lr, max_depth = suggest_hyperparameters(trial)
        mlflow.log_params(trial.params)

        # Fit model
        model = xgb.XGBClassifier(objective='binary:logistic', use_label_encoder=False, seed=123, **params)
        
        # Find the cross-val score
        avg_f1_score_train = cross_val_score(model, X_train, y_train, scoring='f1', cv=3, n_jobs=-1).mean()
        
        # Log f1 score metric using mlflow
        mlflow.log_metric('avg_f1_score_train', avg_f1_score_train)

    # Return the score achieved by the model.
    # This is needed as Optuna needs to know how the suggested hyperparameters are influencing the network loss.
    return avg_f1_score_train

### Main
**Explain the purpose here.**

In [9]:
def main():
    
    # Create the optuna study which shares the experiment name
    study = optuna.create_study(study_name='xgboost-mlflow-optuna', direction='maximize')
    study.optimize(objective, n_trials=100)

    # Print optuna study statistics
    print("\n++++++++++++++++++++++++++++++++++\n")
    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Trial number: ", trial.number)
    print("  Loss (trial value): ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [10]:
main()

[I 2022-04-05 16:31:36,629] A new study created in memory with name: xgboost-mlflow-optuna



********************************



[I 2022-04-05 16:31:40,854] Trial 0 finished with value: 0.5618273557645992 and parameters: {'n_estimators': 273, 'max_depth': 6, 'reg_alpha': 2.1079513592749244, 'reg_lambda': 0.44876101859322737, 'learning_rate': 0.0908627552994408, 'colsample_bytree': 0.47638423461206814, 'subsample': 0.5610745144704892}. Best is trial 0 with value: 0.5618273557645992.



********************************



[I 2022-04-05 16:31:43,355] Trial 1 finished with value: 0.5717347066526504 and parameters: {'n_estimators': 210, 'max_depth': 5, 'reg_alpha': 1.4693844017324358, 'reg_lambda': 1.7307285538843669, 'learning_rate': 0.0536829851641225, 'colsample_bytree': 0.8230448667307504, 'subsample': 0.7054380245586303}. Best is trial 1 with value: 0.5717347066526504.


...


********************************



[I 2022-04-05 16:33:23,301] Trial 98 finished with value: 0.5832822585440746 and parameters: {'n_estimators': 158, 'max_depth': 3, 'reg_alpha': 5.518847096405285, 'reg_lambda': 1.4077580239602525, 'learning_rate': 0.05736370116604319, 'colsample_bytree': 0.7092482598473616, 'subsample': 0.8744077378444821}. Best is trial 30 with value: 0.587959924306742.



********************************



[I 2022-04-05 16:33:23,949] Trial 99 finished with value: 0.5777645022188714 and parameters: {'n_estimators': 110, 'max_depth': 3, 'reg_alpha': 5.313070339841057, 'reg_lambda': 1.359815464506632, 'learning_rate': 0.05284828144188858, 'colsample_bytree': 0.7933529729589548, 'subsample': 0.8346983505638625}. Best is trial 30 with value: 0.587959924306742.



++++++++++++++++++++++++++++++++++

Study statistics: 
  Number of finished trials:  100
Best trial:
  Trial number:  30
  Loss (trial value):  0.587959924306742
  Params: 
    n_estimators: 186
    max_depth: 3
    reg_alpha: 4.264925765715912
    reg_lambda: 1.1552136026096238
    learning_rate: 0.09544177572663988
    colsample_bytree: 0.6383259532921363
    subsample: 0.7480050633488734
